# 1. Intro - imports

In [ ]:
! qi login "https://api.quantum-inspire.com"

In [ ]:
import qiskit.quantum_info as qi
from qi_utilities.algorithms.vqe import construct_hva_circuit, hardware_efficient_vqe
from qi_utilities.experiments.variational_eigensolver import ExecuteVQE
from qiskit_quantuminspire.qi_provider import QIProvider

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
provider = QIProvider()

In [ ]:
provider.backends()

In [ ]:
backend_name = "Tuna-9"
backend = provider.get_backend(name=backend_name)

# 2. VQE workspace

In [ ]:
nr_qubits = 4
J = 1 # in arbitrary units
B = 1 # in arbitrary units

pauli_terms = ['IIXX',
               'IIYY',
               'IIZZ']
               
            #    'IXXI',
            #    'IYYI',
            #    'IZZI',
               
            #    'XXII',
            #    'YYII',
            #    'ZZII',
               
            #    'XIIX',
            #    'YIIY',
            #    'ZIIZ',
               
            #    'IIIZ',
            #    'IIZI',
            #    'IZII',
            #    'ZIII']
pauli_coefficients = [J, J, J]
                    #   J, J, J,
                    #   J, J, J,
                    #   J, J, J,
                      
                    #   B, B, B, B]
hamiltonian_operator = qi.SparsePauliOp(pauli_terms, pauli_coefficients)

superconducting_basis_gates = ['id', 'z', 's', 'sdg', 't', 'tdg', 'x', 'rx', 'y', 'ry', 'cz', 'delay', 'reset']
qubit_list = [0, 2, 4, 1]

In [ ]:
variational_qc = hardware_efficient_vqe(nr_qubits = nr_qubits,
                                        hamiltonian = hamiltonian_operator,
                                        repetitions = 0)

In [ ]:
variational_qc.draw('mpl')

# 3. Overnight code

In [ ]:
variational_qc = hardware_efficient_vqe(nr_qubits = nr_qubits,
                                        hamiltonian = hamiltonian_operator,
                                        repetitions = 0)

execute = ExecuteVQE(variational_qc=variational_qc,
           hamiltonian=hamiltonian_operator,
           hamiltonian_units='a.u.',
           backend=backend,
           qubit_list=qubit_list,
           basis_gates=superconducting_basis_gates,
           nr_shots = 2**14)

In [ ]:
from qi_utilities.utility_functions.readout_correction import measure_ro_assignment_matrix

ro_assignment_matrix = measure_ro_assignment_matrix(backend,
                                                         qubit_list,
                                                         2**14)

In [ ]:
variational_qc = hardware_efficient_vqe(nr_qubits = nr_qubits,
                                        hamiltonian = hamiltonian_operator,
                                        repetitions = 1)

execute = ExecuteVQE(variational_qc=variational_qc,
           hamiltonian=hamiltonian_operator,
           hamiltonian_units='a.u.',
           backend=backend,
           qubit_list=qubit_list,
           basis_gates=superconducting_basis_gates,
           nr_shots = 2**14)

In [ ]:
variational_qc = hardware_efficient_vqe(nr_qubits = nr_qubits,
                                        hamiltonian = hamiltonian_operator,
                                        repetitions = 2)

execute = ExecuteVQE(variational_qc=variational_qc,
           hamiltonian=hamiltonian_operator,
           hamiltonian_units='a.u.',
           backend=backend,
           qubit_list=qubit_list,
           basis_gates=superconducting_basis_gates,
           nr_shots = 2**14)

# Additional testing

In [ ]:
from qiskit import QuantumCircuit, transpile
from qi_utilities.utility_functions.readout_correction import apply_readout_circuit
from qi_utilities.utility_functions.data_handling import StoreProjectRecord
from qi_utilities.utility_functions.circuit_modifiers import prepare_initial_state, apply_pre_measurement_rotations

In [ ]:
# This is a 'theoretical' quantum circuit: the qubit number as well
# as the connectivity of the Tuna backend is being ignored

qubit_0 = 0
qubit_1 = 1
qc = QuantumCircuit(2, 12, name=f'HVA_Ground_State')

meas_idx = 0
for observable in ['IIXX', 'IIYY', 'IIZZ']:
    qc = prepare_initial_state(qc, '11')
    qc.h(qubit_0)
    qc.cx(qubit_0, qubit_1)
    # qc.barrier()
    qc = apply_pre_measurement_rotations(qc, observable, [meas_idx, meas_idx+1])
    meas_idx += 2

qc = apply_readout_circuit(qc, [qubit_0, qubit_1])

In [ ]:
qc.draw('mpl')

In [ ]:
qubit_list = [0, 2]
qc_transpiled = transpile(qc, backend, initial_layout=qubit_list, basis_gates=superconducting_basis_gates)

In [ ]:
qc_transpiled.draw('mpl')

In [ ]:
nr_shots = backend.max_shots
job = backend.run(qc_transpiled, shots=nr_shots, memory = True) # NOTE: memory is set to True in order to return raw data!
result = job.result(timeout = 600)

In [ ]:
StoreProjectRecord(job)

In [ ]:
raw_data_counts

In [ ]:
from qi_utilities.utility_functions.readout_correction import (split_raw_shots, extract_ro_assignment_matrix, get_ro_corrected_multi_probs,
                                                               plot_ro_assignment_matrix)
from qi_utilities.utility_functions.raw_data_processing import get_multi_probs, get_multi_counts

raw_data_shots, ro_mitigation_shots = split_raw_shots(result, qubit_list)
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, qubit_list)

raw_data_counts = get_multi_counts(raw_data_shots, len(qubit_list))
raw_data_probs = get_multi_probs(raw_data_counts)
ro_corrected_probs = get_ro_corrected_multi_probs(raw_data_probs, ro_assignment_matrix, qubit_list)

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, qubit_list)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)

bit_strings = [bit_string for bit_string in raw_data_probs[0]]

ax.bar(bit_strings,
       [raw_data_probs[0][bit_string] for bit_string in raw_data_probs[0]],
       color='blue',
       label='Raw data',
       alpha=0.5, align='edge', width=-0.4)
ax.bar(bit_strings,
       [ro_corrected_probs[0][bit_string] for bit_string in ro_corrected_probs[0]],
       color='orange',
       label='Readout-error-mitigation data',
       align='edge', width=0.4)

ax.set_xlabel("Bit strings")
ax.set_ylabel("Probabilities")
ax.set_title(f"HVA ground state preparation\n{backend_name} processor\nQubits Q{qubit_list[0]} - Q{qubit_list[1]}")

ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0])
ax.set_ylim(0.0, 1.05)
ax.legend()

plt.grid(axis='y')
plt.show()

In [ ]:
ro_corrected_probs

In [ ]:
from qi_utilities.utility_functions.raw_data_processing import observable_expectation_values_Z_basis

observable_expectation_values_Z_basis([ro_corrected_probs[2]], 'ZZ')

In [ ]:
observable_expectation_values_Z_basis(ro_corrected_probs, 'ZZ')

In [ ]:
observable_expectation_values_Z_basis([ro_corrected_probs[2]], 'ZZ')

In [ ]:
ro_corrected_probs[1]

In [ ]:
ro_corrected_probs